# Задание 1

In [41]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('data/2ch_corpus.txt').read()[:10000]
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные

news = open('data/lenta.txt').read()[:10000]

In [42]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [43]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [44]:
from collections import Counter

In [45]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [46]:
sentences_dvach = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [47]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
threegrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    threegrams_dvach.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
threegrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    threegrams_news.update(ngrammer(sentence, n=3))

In [37]:
matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigrams_dvach = list(bigrams_dvach)
bigrams2id_dvach = {bigram:i for i, bigram in enumerate(id2bigrams_dvach)}



for ngram in threegrams_dvach:
    word1, word2, word3 = ngram.split()
    
    key_bi = word1 + ' ' + word2

    
    matrix_dvach[bigrams2id_dvach[key_bi]][word2id_dvach[word3]] =  (threegrams_dvach[ngram]/
                                                                     bigrams_dvach[key_bi])


In [38]:
# создадим матрицу вероятностей перейти из биграммы в другое слово
matrix_news = np.zeros((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigrams_news = list(bigrams_news)
bigrams2id_news = {bigram:i for i, bigram in enumerate(id2bigrams_news)}


# вероятность расчитываем точно также
for ngram in threegrams_news:
    word1, word2, word3 = ngram.split()
    
    key_bi = word1 + ' ' + word2
    
    matrix_news[bigrams2id_news[key_bi]][word2id_news[word3]] =  (threegrams_news[ngram]/
                                                                     bigrams_news[key_bi])



In [39]:
def generate(matrix, id2word, word2id, id2bigram, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        
        current_bigram = id2bigram[current_idx]
        current_second_word = current_bigram.split()[1]
        
        next_bigram = f'{current_second_word} {id2word[chosen]}'
        
        if id2word[chosen] == '<end>':
            current_idx = bigram2id['<start> <start>']
        else:
            current_idx = bigram2id[next_bigram]
    
    return ' '.join(text)

In [40]:
print(generate(matrix_news, id2word_news, word2id_news, id2bigrams_news, bigrams2id_news).replace('<end>', '\n'))

много раз пробирался он через линии германских войск неся на себе спрятанными в ошейнике шифрованные депеши 
 с места происшествия запросили 20 машин скорой помощи 
 агентство итар-тасс в сообщении от 21.15 со ссылкой на источники в гувд москвы говорит только о 30 раненых в том числе о двух пострадавших в результате этого взрыва может составить до ста человек 
 среди причин происшедшего называют террористический акт связанный с последними событиями в дагестане однако по сообщению орт не исключается и версия об аварии взорвался игровой автомат 
 когда показывался аэроплан-птица красиво и вольно паривший в воздухе австрийцы указывали — das ist


In [34]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, bigrams2id_dvach).replace('<end>', '\n'))

второй опенинг параша но дискасс 
 так и должно быть то ближе к середине я не понимал почему он не становится брутальным дротикометателем 
 и наверное пока единственный 
 ну к сайто у меня так ебать кожа на ебле шелушится шо делать где можно скачать сей тайтл 
 цундере не нужны 
 тоха cinderella cage принцесса лучше всех 
 и мне и гатари нравятся как и тамошняя манера подачи просто я считаю что автор мысль доносил на примере слабоватых примеров вот и все 
 даешь хенриетту с сайто на свадьбу и хэппи энд смотрел 
 освятил шедевром японской анимации эта


# Задание 2

In [54]:
import itertools
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation
from nltk.corpus import stopwords
import gensim

In [49]:
stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [51]:
corpus = open('data/lenta.txt').read()

In [52]:
corpus = preprocess(corpus)

In [75]:
def scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    if bigram_count >= min_count:
        corpus_word_count = float(corpus_word_count)
        pa = worda_count / corpus_word_count
        pb = wordb_count / corpus_word_count
        pab = bigram_count / corpus_word_count
        try:
            return np.log(pab / (pa * pb))
        except ValueError:
            return -100000
    else:
        return -100000
    
    
    return 

In [76]:
corpus[:3]

[['бой', 'сопоцкина', 'друскеник', 'закончиться', 'отступление', 'германец'],
 ['неприятель',
  'приблизиться',
  'север',
  'осовца',
  'начать',
  'артиллерийский',
  'борьба',
  'крепость'],
 ['артиллерийский', 'бой', 'принимать', 'участие', 'тяжёлый', 'калибр']]

In [86]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=0)
p = gensim.models.phrases.Phraser(ph)

In [87]:
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=0)
p2 = gensim.models.phrases.Phraser(ph2)

In [88]:
p2[p[corpus[333]]]

['установить',
 'взрыв_произойти',
 'третий',
 'уровнечетвертый',
 'ярус',
 'комплекс',
 'зал',
 'игровой_автомат']

In [95]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=5)
p = gensim.models.phrases.Phraser(ph)

In [96]:
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=5)
p2 = gensim.models.phrases.Phraser(ph2)

In [97]:
p2[p[corpus[333]]]

['установить',
 'взрыв',
 'произойти',
 'третий',
 'уровнечетвертый',
 'ярус',
 'комплекс',
 'зал',
 'игровой_автомат']

In [98]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=10)
p = gensim.models.phrases.Phraser(ph)

In [99]:
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=10)
p2 = gensim.models.phrases.Phraser(ph2)

In [100]:
p2[p[corpus[333]]]

['установить',
 'взрыв',
 'произойти',
 'третий',
 'уровнечетвертый',
 'ярус',
 'комплекс',
 'зал',
 'игровой',
 'автомат']

Из примера видно, что при пороговом значении 0 были обе биграммы, при значении 5 осталась одна, а вторая не прошла, а при значении 10 не прошли обе биграммы